In [ ]:
import os
import sys
import serial
import threading
import datetime
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import re

# Column headers for the Google Spreadsheet
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets scope and client initialization
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Global stop event for threads
stop_event = threading.Event()

# Main GUI Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Data Monitor - Raspberry Pi")

        # Variables for the GUI
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.data_queue = queue.Queue()
        self.threads = []
        self.device_mappings = []
        self.port_widgets = {}

        # Google Sheets client
        self.gspread_client = None

        # Setup the GUI interface
        self.setup_gui()

    def setup_gui(self):
        # Experimenter name input
        tk.Label(self.root, text="Experimenter Name:", font=("Helvetica", 10, "bold")).grid(column=0, row=0, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=2, pady=2)

        # Experiment name input
        tk.Label(self.root, text="Experiment Name:", font=("Helvetica", 10, "bold")).grid(column=2, row=0, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.W, padx=2, pady=2)

        # JSON file path
        tk.Label(self.root, text="Google API JSON File:", font=("Helvetica", 10, "bold")).grid(column=0, row=1, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path)
        self.json_entry.grid(column=1, row=1, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(self.root, text="Browse", command=self.browse_json)
        self.browse_json_button.grid(column=2, row=1, padx=5, pady=10)

        # Google Spreadsheet ID
        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Helvetica", 10, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id)
        self.spreadsheet_entry.grid(column=1, row=2, sticky=tk.W, padx=2, pady=2)

        # Start button
        self.start_button = tk.Button(self.root, text="START", font=("Helvetica", 15, "bold"), bg="green", fg="white", command=self.start_logging)
        self.start_button.grid(column=1, row=3, padx=5, pady=10)

        # Stop button
        self.stop_button = tk.Button(self.root, text="STOP(SAVE & QUIT)", font=("Helvetica", 15, "bold"), bg="red", fg="white", command=self.stop_logging)
        self.stop_button.grid(column=2, row=3, padx=5, pady=10)

        # Create GUI for each port
        self.setup_ports()

    def browse_json(self):
        # Open a file dialog to select JSON file
        self.json_path.set(filedialog.askopenfilename(title="Select JSON File"))

    def setup_ports(self):
        # Port panel setup for tracking status and data
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.root, text=port_name)
            frame.grid(column=idx, row=4, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", font=("Helvetica", 14, "italic"), foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=40, height=10)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            # Store widgets for later updates
            self.port_widgets[port_name] = {
                'frame': frame,
                'status_label': status_label,
                'text_widget': text_widget
            }

    def start_logging(self):
        # Initialize Google Sheets API and verify connection
        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            print(f"Connected to Google Spreadsheet: {self.spreadsheet_id.get()}")
        except Exception as e:
            messagebox.showerror("Error", f"Error setting up Google Sheets API: {e}")
            return

        # Start reading from FED3 devices and log data to Google Sheets
        self.start_data_collection()

    def start_data_collection(self):
        # Automatically detect and handle the connected FED3 devices using USB ports
        self.device_mappings = self.get_device_mappings_by_usb_port()

        # Start a separate thread for each FED3 device
        for mapping in self.device_mappings:
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            q = self.port_widgets[port_identifier]['text_widget']  # Assign queue to text widget
            t = threading.Thread(target=self.read_from_port, args=(serial_port, f"Port_{port_identifier}", q))
            t.daemon = True
            t.start()
            self.threads.append(t)

    def get_device_mappings_by_usb_port(self):
        # Automatically map devices to ports
        device_mappings = []
        usb_port_mapping = {
            'usb-0:1.1': 'Port 1',
            'usb-0:1.2': 'Port 2',
            'usb-0:1.3': 'Port 3',
            'usb-0:1.4': 'Port 4',
        }
        for symlink in os.listdir('/dev/serial/by-path/'):
            symlink_path = os.path.join('/dev/serial/by-path/', symlink)
            serial_port = os.path.realpath(symlink_path)
            if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
                usb_port_path = self.get_usb_port_path_from_symlink(symlink)
                port_identifier = usb_port_mapping.get(usb_port_path)
                if port_identifier:
                    device_mappings.append({
                        'serial_port': serial_port,
                        'port_identifier': port_identifier,
                    })
                    self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
        return device_mappings

    def get_usb_port_path_from_symlink(self, symlink):
        match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
        return match.group() if match else None

    def read_from_port(self, serial_port, worksheet_name, text_widget):
        # Open the serial port and connect to Google Spreadsheet
        try:
            ser = serial.Serial(serial_port, 115200, timeout=1)
            spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            sheet = self.get_or_create_worksheet(spreadsheet, worksheet_name)

            while True:
                data = ser.readline().decode('utf-8').strip()
                data_list = data.split(",")
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                data_list = data_list[1:]  # Skip FED device timestamp

                if len(data_list) == len(column_headers) - 1:
                    sheet.append_row([timestamp] + data_list)
                    self.data_queue.put([timestamp] + data_list)

                    # Update the text widget for the corresponding port
                    text_widget.insert(tk.END, f"Data logged: {data_list}\n")
                    text_widget.see(tk.END)
        except Exception as e:
            messagebox.showerror("Error", f"Error reading from port {serial_port}: {e}")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(column_headers)
            return sheet

    def stop_logging(self):
        # Stop all running threads and cleanup
        stop_event.set()
        for t in self.threads:
            t.join()

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


Connected to Google Spreadsheet: 1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk
